<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [8]:
import tensorflow

In [28]:
##### Your Code Here #####
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [15]:
# read in data
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')

In [16]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [17]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [20]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [18]:
# drop the id, leakage
df = df.drop(columns=['customerID'])

In [21]:
# total charges is a float not an object
def to_float(i):
    j = i.strip()
    if j == '':
        return 0
    else:
        return float(j)
    
df['TotalCharges'] = df['TotalCharges'].apply(to_float)

In [37]:
def wrangle(df, target, features):
    # train test split
    train, test = train_test_split(df)
    
    X = train[features]
    y = train[target]
    
    X_test = test[features]
    y_test = test[target]
    
    # encode categorical columns and scale num columns
    cols = X.columns
    num_cols = X._get_numeric_data().columns
    cat_cols = list(set(cols) - set(num_cols))
    
    X_cat = X[cat_cols]
    X_num = X[num_cols]
    X_test_cat = X_test[cat_cols]
    X_test_num = X_test[num_cols]
    
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    X_cat = encoder.fit_transform(X_cat)
    X_test_cat = encoder.fit_transform(X_test_cat)
    
    scaler = StandardScaler()
    
    X_num = pd.DataFrame(
       scaler.fit_transform(X_num),
       columns=num_cols)
    X_test_num = pd.DataFrame(
       scaler.fit_transform(X_test_num),
       columns=num_cols)
    X_cat = pd.DataFrame(X_cat, columns=encoder.get_feature_names())
    X = X_num.join(X_cat)
    X_test_cat = pd.DataFrame(X_test_cat, columns=encoder.get_feature_names())
    X_test = X_test_num.join(X_test_cat)
    
    # convert yes/no to num boolean
    y = y.replace({'No': 0, 'Yes': 1})
    y_test = y_test.replace({'No': 0, 'Yes': 1})
    
    print(X.shape, X_test.shape, y.shape, y_test.shape)
    return X, y, X_test, y_test

target = 'Churn'
features = df.columns.drop(['Churn'])
X, y, X_test, y_test = wrangle(df, target, features)

(5282, 45) (1761, 45) (5282,) (1761,)


In [38]:
# baseline
1 - y.mean()

0.7351382052252935

In [39]:
# model
model = Sequential([
                    Dense(128, activation='relu', input_dim=45),
                    Dense(128, activation='relu'),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5888      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 26,561
Trainable params: 26,561
Non-trainable params: 0
_________________________________________________________________


In [40]:
from tensorflow.keras.callbacks import EarlyStopping

stop = EarlyStopping(monitor='val_accuracy', min_delta=.001, patience=2)
model.fit(X, y,
         validation_data=(X_test, y_test),
         epochs=100,
         batch_size=None,
         callbacks=[stop])

Train on 5282 samples, validate on 1761 samples
Epoch 1/100
5282/5282 [==============================] - 2s 289us/sample - loss: 0.4435 - accuracy: 0.7870 - val_loss: 0.4175 - val_accuracy: 0.7995
Epoch 2/100
5282/5282 [==============================] - 1s 99us/sample - loss: 0.4186 - accuracy: 0.8048 - val_loss: 0.4145 - val_accuracy: 0.8012
Epoch 3/100
5282/5282 [==============================] - 1s 117us/sample - loss: 0.4131 - accuracy: 0.8061 - val_loss: 0.4199 - val_accuracy: 0.8001
Epoch 4/100
5282/5282 [==============================] - 0s 92us/sample - loss: 0.4091 - accuracy: 0.8042 - val_loss: 0.4118 - val_accuracy: 0.8030
Epoch 5/100
5282/5282 [==============================] - 0s 89us/sample - loss: 0.4033 - accuracy: 0.8111 - val_loss: 0.4151 - val_accuracy: 0.8018
Epoch 6/100
5282/5282 [==============================] - 1s 137us/sample - loss: 0.3995 - accuracy: 0.8129 - val_loss: 0.4155 - val_accuracy: 0.8030


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?

In [ ]:
# stretch goals
